In [1]:
# Import Dependencies
import pandas as pd 
import json
import requests
import numpy as np
import datetime
from datetime import timedelta


In [2]:
# Read the CSV file from AWS to Pandas Dataframe
url = "https://finalproject-3.s3.us-west-1.amazonaws.com/2017.csv"

df = pd.read_csv(url)

df.head()

FL_DATE OP_CARRIER  ...  LATE_AIRCRAFT_DELAY Unnamed: 27
0  2017-01-01         AA  ...                  0.0         NaN
1  2017-01-01         AA  ...                  0.0         NaN
2  2017-01-01         AA  ...                 35.0         NaN
3  2017-01-01         AA  ...                  0.0         NaN
4  2017-01-01         AA  ...                  0.0         NaN

[5 rows x 28 columns]

In [3]:
# Filter the data for Tucson Airport
df1 = df.loc[df['ORIGIN'] == 'TUS']



# df1.to_csv('tus.csv')
# files.download('tus.csv')

# Remove the Unnamed columns which as no values
tus_df = df1.drop(['Unnamed: 27'], axis = 1)


In [4]:
# Fill all NaN with 0
from google.colab import files
tus_df1 = tus_df.fillna(0)



In [13]:
# if dep_time = 0 the File was cancelled

tus_df['CRS_DEP_HM'] = pd.to_datetime(tus_df['CRS_DEP_TIME'], format = '%H%M').dt.time

# tus_df1 = tus_df1[(tus_df1['DEP_TIME'] !=8) & (tus_df1['DEP_TIME'] !=4) & (tus_df1['DEP_TIME'] !=0)]
# tus_df1['DEP_TIME_HM'] = pd.to_datetime(tus_df1['DEP_TIME'], format = '%H%M').dt.time


tus_df['CRS_ARR_HM'] = pd.to_datetime(tus_df['CRS_ARR_TIME'], format = '%H%M').dt.time

# tus_df1 = tus_df1[(tus_df1['ARR_TIME'] !=0) &  (tus_df1['ARR_TIME'] !=5)]
# tus_df1['ARR_TIME_HM'] = pd.to_datetime(tus_df1['ARR_TIME'], format = '%H%M').dt.time

# tus_df1 = tus_df1[(tus_df1['WHEELS_OFF'] !=0) & (tus_df1['WHEELS_OFF'] !=3)]
# tus_df1['WHEELS_OFF_HM'] = pd.to_datetime(tus_df1['WHEELS_OFF'], format = '%H%M').dt.time

tus_df
# tus_df1.to_csv('tus.csv')
# files.download('tus.csv')

FL_DATE OP_CARRIER  ...  CRS_DEP_HM CRS_ARR_HM
73       2017-01-01         AA  ...    13:22:00   17:48:00
206      2017-01-01         AA  ...    06:05:00   09:16:00
323      2017-01-01         AA  ...    13:55:00   17:07:00
851      2017-01-01         EV  ...    11:25:00   14:53:00
1227     2017-01-01         OO  ...    10:50:00   12:57:00
...             ...        ...  ...         ...        ...
5672833  2017-12-31         EV  ...    05:25:00   08:57:00
5673130  2017-12-31         EV  ...    15:00:00   18:28:00
5673149  2017-12-31         EV  ...    12:08:00   15:36:00
5673413  2017-12-31         UA  ...    07:30:00   10:54:00
5674286  2017-12-31         UA  ...    14:40:00   16:42:00

[15406 rows x 29 columns]

In [14]:
# Add Day of the week

tus_df['FL_DATE'] = pd.to_datetime(tus_df['FL_DATE'], format = '%Y/%m/%d')
tus_df['day_of_week'] = tus_df['FL_DATE'].dt.day_name()
tus_df

# tus_df1.to_csv('tus.csv')
# files.download('tus.csv')



FL_DATE OP_CARRIER  ...  CRS_ARR_HM day_of_week
73      2017-01-01         AA  ...    17:48:00      Sunday
206     2017-01-01         AA  ...    09:16:00      Sunday
323     2017-01-01         AA  ...    17:07:00      Sunday
851     2017-01-01         EV  ...    14:53:00      Sunday
1227    2017-01-01         OO  ...    12:57:00      Sunday
...            ...        ...  ...         ...         ...
5672833 2017-12-31         EV  ...    08:57:00      Sunday
5673130 2017-12-31         EV  ...    18:28:00      Sunday
5673149 2017-12-31         EV  ...    15:36:00      Sunday
5673413 2017-12-31         UA  ...    10:54:00      Sunday
5674286 2017-12-31         UA  ...    16:42:00      Sunday

[15406 rows x 30 columns]

In [15]:
# Airport Codes
url_airport = "https://finalproject-3.s3.us-west-1.amazonaws.com/airports_codes.csv"

airport = pd.read_csv(url_airport)

airport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9235 entries, 0 to 9234
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   AirportCode    9235 non-null   object
 1   CityName       9235 non-null   object
 2   WorldAreaCode  9235 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 216.6+ KB


In [16]:
merge_dest_df = pd.merge(airport, tus_df, left_on='AirportCode', right_on='DEST', how='inner')

In [17]:
merge_dest_origin_df = pd.merge(airport, merge_dest_df, left_on='AirportCode', right_on='ORIGIN', how='inner')
merge_dest_origin_df

AirportCode_x                           CityName_x  ...  CRS_ARR_HM day_of_week
0               TUS  TUCSON, ARIZONA, USA                 ...    20:03:00      Sunday
1               TUS  TUCSON, ARIZONA, USA                 ...    11:29:00      Monday
2               TUS  TUCSON, ARIZONA, USA                 ...    20:05:00      Monday
3               TUS  TUCSON, ARIZONA, USA                 ...    11:29:00     Tuesday
4               TUS  TUCSON, ARIZONA, USA                 ...    20:05:00     Tuesday
...             ...                                  ...  ...         ...         ...
15401           TUS  TUCSON, ARIZONA, USA                 ...    14:29:00      Friday
15402           TUS  TUCSON, ARIZONA, USA                 ...    10:18:00    Saturday
15403           TUS  TUCSON, ARIZONA, USA                 ...    14:29:00    Saturday
15404           TUS  TUCSON, ARIZONA, USA                 ...    10:18:00      Sunday
15405           TUS  TUCSON, ARIZONA, USA                 ...    14:29:00      Sunday

[15406 rows x 36 columns]

In [18]:
# Rename Columns
rename_dest_origin_df = merge_dest_origin_df.rename(columns = {'CityName_x': 'origin_city_name',
                             'CityName_y': 'dest_city_name'})

rename_dest_origin_df

AirportCode_x                     origin_city_name  ...  CRS_ARR_HM day_of_week
0               TUS  TUCSON, ARIZONA, USA                 ...    20:03:00      Sunday
1               TUS  TUCSON, ARIZONA, USA                 ...    11:29:00      Monday
2               TUS  TUCSON, ARIZONA, USA                 ...    20:05:00      Monday
3               TUS  TUCSON, ARIZONA, USA                 ...    11:29:00     Tuesday
4               TUS  TUCSON, ARIZONA, USA                 ...    20:05:00     Tuesday
...             ...                                  ...  ...         ...         ...
15401           TUS  TUCSON, ARIZONA, USA                 ...    14:29:00      Friday
15402           TUS  TUCSON, ARIZONA, USA                 ...    10:18:00    Saturday
15403           TUS  TUCSON, ARIZONA, USA                 ...    14:29:00    Saturday
15404           TUS  TUCSON, ARIZONA, USA                 ...    10:18:00      Sunday
15405           TUS  TUCSON, ARIZONA, USA                 ...    14:29:00      Sunday

[15406 rows x 36 columns]

In [19]:
# Drop colums not needed.
tus_final_df = rename_dest_origin_df.drop(['AirportCode_x', 'WorldAreaCode_x', 'AirportCode_y', 'WorldAreaCode_y'], axis = 1)
tus_final_df

# Export the final DF
tus_final_df.to_csv('2017_TUS.csv', index = False)
files.download('2017_TUS.csv')

tus_final_df



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

origin_city_name  ... day_of_week
0      TUCSON, ARIZONA, USA                 ...      Sunday
1      TUCSON, ARIZONA, USA                 ...      Monday
2      TUCSON, ARIZONA, USA                 ...      Monday
3      TUCSON, ARIZONA, USA                 ...     Tuesday
4      TUCSON, ARIZONA, USA                 ...     Tuesday
...                                    ...  ...         ...
15401  TUCSON, ARIZONA, USA                 ...      Friday
15402  TUCSON, ARIZONA, USA                 ...    Saturday
15403  TUCSON, ARIZONA, USA                 ...    Saturday
15404  TUCSON, ARIZONA, USA                 ...      Sunday
15405  TUCSON, ARIZONA, USA                 ...      Sunday

[15406 rows x 32 columns]